# Notebook for running calibration / training

## Import settings from settings notebook

In [1]:
%run MP1_settings.ipynb

## Load file with the temperature information for all traning data files
The file contains the temperature for all files we had available, but later only the temperatures of the files in the training data folder is used during training. You don't have to delete unused file names from the list, just delete the file from the folder.

In [2]:
TEMPERATURE_FILE = r'data\train_data\temperature.dat' # Name of file

temperature_list = pd.read_csv(TEMPERATURE_FILE, delimiter=r',', decimal='.') #Loading file

## Load all files from training data folder and lookup the temperature information

In [3]:
line = []
freq = []
temperature = []

file_list = glob.glob(r'data\train_data\*.csv')
for filename in file_list:
    df = pd.read_csv (filename, delimiter=r'\s+,', decimal=',', skiprows=1, header=None, names=colnames, engine='python')
    line.append(np.array(df[Y_NAME]))
    freq.append(np.array(df[X_NAME]))
    temperature_given = (filename.split('train_data\\')[-1]==temperature_list['file'])
    temperature.append(*np.array(temperature_list[temperature_given]['temperature']))

n = len(freq)
n_freq = len(freq[0])
line = np.array(line)
freq = np.array(freq)
temperature = np.array(temperature)

print('{} files loaded.'.format(n))

21 files loaded.


# Selecting the Baseline
The selection of the baseline is somewhat arbitrary. We just select the first (by name) file with the lowest temperature. Setting $i=0$ will select just the first (by name) file as the baseline.

In [4]:
#Basislinie wird festgelegt
i = np.array(np.where(temperature == temperature.min())).flatten()[0] # in this case: first file with the lowest temperature
# i = 0 # in this case: first file

baseline = []# y-values
basefreq = []# x-values

for f in range(n_freq):
    if (FREQ_MIN<=freq[i,f])&(freq[i,f]<=FREQ_MAX):
        baseline.append(np.mean(line[:10,f]))
        basefreq.append(freq[i,f])
        
baseline = np.array(baseline)
basefreq = np.array(basefreq)

n_freq_base = len(basefreq)

print('File\n{}\nis the baseline.'.format(file_list[i]))

File
data\train_data\plate_4.csv
is the baseline.


In [5]:
freq_shifted = np.zeros((n,n_freq))

for i in range(n):
    def min_func(p):
        line_int = interpolate.interp1d(horizontal_shift(freq[i], p), line[i], kind=INT_METHOD, bounds_error=False)(basefreq)
        return 1 - CC_btw(line_int, baseline)

    popt = minimize(min_func, hor_PARAM_INIT, method=MIN_METHOD, tol=1e-12, bounds=hor_PARAM_RANGE)

    freq_shifted[i] = horizontal_shift(freq[i], popt.x)

    clear_output(wait=True)
    print(f'Horizontal shift assessment')
    print(f'{i+1}/{n}')

Horizontal shift assessment
21/21


In [6]:
line_fit = np.zeros((n,n_freq_base))

for i in range(n):
    line_fit[i] = interpolate.interp1d(freq_shifted[i], line[i], kind=INT_METHOD, bounds_error=False)(basefreq)

In [7]:
vert_shift_spectrum = np.zeros((ver_DIM, n_freq_base)) 
vert_shift_spectrum[0] = baseline

In [8]:
for f in range(n_freq_base):
    
    def min_func(p):
        return RMSD_btw(np.tile(baseline[f], n), vertical_shift(line_fit[:,f], p, temperature))

    popt = minimize(min_func, vert_shift_spectrum[:,f], method=MIN_METHOD, tol=1e-12)
    vert_shift_spectrum[:,f] = popt.x
    
    clear_output(wait=True)
    print('Vertical shift assessment')
    print('Frequency: {} Hz'.format(basefreq[f]))

Vertical shift assessment
Frequency: 79800 Hz


# Save Parameters and Baseline

In [9]:
header = ['basefreq', 'baseline']
for k in range(ver_DIM):
    header.append(f'parameter_{k}')

with open(parameter_path, 'w', encoding='UTF8') as f:
    writer = csv.writer(f, delimiter =',')
    f.truncate(0)
    # Write the header
    writer.writerow(header)
    
    for f in range(n_freq_base):
        data = [basefreq[f], baseline[f], *vert_shift_spectrum[:,f]]
        # Write data
        writer.writerow(data)
        
print('Parameters and Baseline saved to {}'.format(parameter_path))

Parameters and Baseline saved to data\trainable_parameters.dat
